# file structure

> How to store predefined file structure of objects and check existence


In [ ]:
# | default_exp core.files.structure

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# | export
# | hide

from __future__ import annotations

import json
import logging
import os
import re
from pathlib import Path
from typing import Any, Callable, Literal

import pandas as pd
from dotenv import load_dotenv
from numpy import rec

from ds_contrib.core.paths import Directory, PathLike, pathify
from ds_contrib.core.utils import Iterifiable, listify
from ds_contrib.tools.io.gscloud import GSBrowser, GSBrowserContext
from ds_contrib.core.paths import handle_existing_path

In [ ]:
# | export
# | hide

logger = logging.getLogger(__name__)

## File Structure node

Building file structure from a json file with structure and list of paths


In [ ]:
# | hide

CWD = Path.cwd()
REPO_DIR = Path(*CWD.parts[: CWD.parts.index("ds_contrib") + 1])
CONFIGS_DIR = REPO_DIR / "configs"
ENV_DIR = CONFIGS_DIR / "env/local"

with open(CONFIGS_DIR / "storage/gscloud/projects_vars.json") as f:
    projects = json.load(f)

# choose project
project = projects["dev"]
env_path = Path(ENV_DIR / f'{project["env"]}_roadly.env')

_ = load_dotenv(env_path)  # read local .env file
google_app_creds = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
roadly_cookie = os.getenv("ROADLY_COOKIE")
print(f"Initial configuration has finished:\nProject: {project}")

Initial configuration has finished:
Project: {'project': 'roadly-project-dev', 'env': 'dev', 'coldline_name': 'standard'}


In [ ]:
# | hide

RECSLAM_DATA_PATH = "roadly-dev-videos/Antalya/2022-02-21_13-18-38_4453/"

with open(CONFIGS_DIR / "storage/recslam/recslam_structure.json", "r") as f:
    RECSLAM_STRUCTURE = json.load(f)

In [ ]:
# | export
# | hide


class FSNode:
    def __init__(
        self,
        name: str | None = None,
        children: dict[str, FSNode] | Literal["pattern"] | None = None,
        path: str | None = None,
        description: str | None = None,
        exists: bool | None = None,
        type: Literal["file", "group", "pattern"] | None = None,
        prefix: str | None = None,
        **meta,
    ):
        """File structure node class.

        Used to represent a file structure in a hierarchical way. Each node can be a file, a group of files (logical entity) or a pattern for searching files.

        Parameters
        ----------
        name : str | None, optional
            name of the node, by default None
        children : dict[str, FSNode] | Literal[&quot;pattern&quot;] | None, optional
            child nodes, by default None
        path : str | None, optional
            path of the node from the root of the file structure, by default None
        description : str | None, optional
            description of the node, by default None
        exists : bool | None, optional
            whether the node indicated in the file structure exists in a parsed files (look at `parse_files` method),
            None means that the node was not parsed yet, by default None
        type : Literal[&quot;file&quot;, &quot;group&quot;, &quot;pattern&quot;] | None, optional
            type of the node, types are: `file`, `group`, `pattern`, by default None
        prefix : str | None, optional
            prefix of the node, by default None
        **meta :
            additional meta information about the node, e.g. `{"_file_description": "description of the file"}`
        """
        self.name = name
        self.prefix = prefix
        self._children: dict[tuple[str, str], FSNode] | Literal[
            "pattern"
        ] | None = children
        self.path = path
        self.description = description
        self.exists: bool | None = exists
        self.type: Literal["file", "group", "pattern"] | None = type
        self.meta = meta

    def attach_meta(
        self,
        key: str,
        value: Any,
        existing_handling: Literal["raise", "skip", "overwrite"] = "overwrite",
    ):
        """Attach meta information to the node.

        Parameters
        ----------
        key : str
            meta key to attach to the node, if the meta key starts with `_` it will be hidden in `df` property
        value : Any
            meta value to attach to the node
        existing_handling : Literal['raise', 'skip', 'overwrite'], optional
            how to handle existing meta keys:
            - `raise` raises an error if the key already exists
            - `skip` skips the key if it already exists
            - `overwrite` overwrites the key if it already exists
        filtration of params must be provided by external code
        """
        # TODO[High]: add existing_handling strategies
        self.meta[key] = value

    @property
    def full_names(self):
        return [f"{prefix}/{name}" for prefix, name in self.df.index]

    @property
    def is_file(self):
        return self.type == "file"

    def parse_files(self, paths: Iterifiable[PathLike]) -> set[PathLike]:
        """Main method for parsing a list of files and marking the nodes that exist in the file structure.

        The method is recursive and parses all children nodes.
        WARNING: some nodes may not be parsed if they do not exist in the file structure,
        some nodes' contents may be changed e.g. for `pattern` nodes.

        Parameters
        ----------
        paths : Iterifiable[PathLike]
            iterable of paths to parse or a single path

        Returns
        -------
        set[PathLike]
            set of paths that were not parsed due to the fact that they do not exist in the file structure

        Raises
        ------
        ValueError
            If the node type is not one of `file`, `group`, `pattern`.
        """
        remained_paths = set(listify(paths, none_handlings="empty"))
        # first parse all files
        if self.type == "file":
            if self.path in remained_paths:
                self.exists = True
                remained_paths.remove(self.path)
            else:
                self.exists = False
        elif self.type == "group":
            for child in self._children.values():
                remained_paths = child.parse_files(remained_paths)
        elif self.type == "pattern":
            deleted_paths = []
            for path in remained_paths:
                if re.match(self.path, path):
                    name = Path(path).stem
                    prefix = self._prefix_generator(self)
                    self._children[(".", name)] = self.__class__(
                        name,
                        path=path,
                        type="file",
                        description=self.meta["_file_description"],
                        exists=True,
                        prefix=prefix,
                    )
                    deleted_paths.append(path)
            for path in deleted_paths:
                remained_paths.remove(path)
        else:
            raise ValueError(f"Unknown node type {self.type}")
        return remained_paths

    @staticmethod
    def _prefix_generator(parent_node: FSNode):
        return (
            f"{parent_node.prefix}/{parent_node.name}"
            if parent_node.prefix
            else (parent_node.name if parent_node.name != "root_node" else "")
        )

    @classmethod
    def _parse_group(cls, d: dict, parent_node: FSNode):
        parent_node.description = d.get("desc")
        parent_node.path = d.get("path")
        parent_node.type = "group"
        prefix = cls._prefix_generator(parent_node)
        parent_node._children = {
            (".", k): cls._parse_dict(v, cls(name=k, prefix=prefix))
            for k, v in d["nodes"].items()
        }
        return parent_node

    @classmethod
    def _parse_pattern(cls, d: dict, parent_node: FSNode):
        parent_node.type = "pattern"
        parent_node.description = d.get("desc")
        parent_node.meta["_file_description"] = d["path_pattern"].get("desc")
        parent_node._children = {}
        parent_node.path = d["path_pattern"]["path"]
        return parent_node

    @staticmethod
    def _parse_file_node(d: dict, parent_node: FSNode):
        parent_node.description = d.get("desc")
        parent_node.path = d.get("path")
        parent_node.type = "file"
        parent_node._children = None
        return parent_node

    @classmethod
    def _parse_dict(cls, d: dict, parent_node: FSNode):
        if "path_pattern" in d:
            return cls._parse_pattern(d, parent_node)
        elif "nodes" in d:
            return cls._parse_group(d, parent_node)
        else:
            return cls._parse_file_node(d, parent_node)

    @classmethod
    def from_dict(cls, d: dict) -> FSNode:
        """Generate a `FSNode` with hierarchical structure from a dictionary.

        Parameters
        ----------
        d : dict
            dictionary with hierarchical structure of the file structure, e.g. as in `recslam_structure.json`

        Returns
        -------
        FSNode
            root `FSNode` object with hierarchical structure of the file structure
        """
        root_node = FSNode(name="root_node", description="root_node", type="group")
        return cls._parse_dict(d, root_node)

    def _to_dict(self):
        return {
            # "name": self.name,
            "path": self.path,
            "description": self.description,
            "type": self.type,
            "exists": self.exists,
            **{f"meta[{k}]": v for k, v in self.meta.items() if not k.startswith("_")},
        }

    def __repr__(self):
        s = f"{self.__class__.__name__}: [{self.name}]\n"
        if self._children:
            s += repr({k: v._to_dict() for k, v in self._children.items()})
        return s

    @property
    def df(self) -> pd.DataFrame:
        """Return a `pd.DataFrame` with all nodes in the file structure as rows.

        Returns
        -------
        pd.DataFrame
        """
        # TODO[HIGH]: Add caching for df and reset cache on change
        if self._children:
            df = pd.DataFrame.from_dict(
                {(v.prefix, v.name): v._to_dict() for v in self._children.values()},
                orient="index",
            )
        else:
            df = pd.DataFrame.from_dict(
                {(self.prefix, self.name): self._to_dict()}, orient="index"
            )
        df.index.set_names(["prefix", "name"], inplace=True)
        return df

    def _repr_html_(self):
        s = f"<b>{self.__class__.__name__}: [{self.type}]{'(empty)' if self.type != 'file' and (self._children is None or len(self._children) == 0) else ''} {self.name}</b>: {self.description}<br><br>"
        s += self.df._repr_html_()
        return s

    def _get_files(
        self,
        files: dict[str, FSNode] = None,
        only_exists: bool = False,
        max_recursion_depth: int = 999,
    ):
        if self.type == "file":
            if not only_exists or self.exists:
                # key = f"{self.prefix}/{self.name}" if self.prefix else self.name
                files[self.prefix, self.name] = self
        elif self.type in ("group", "pattern"):
            if max_recursion_depth > 0:
                for child in self._children.values():
                    child._get_files(
                        files,
                        only_exists,
                        max_recursion_depth=max(max_recursion_depth - 1, 0),
                    )
        else:
            raise ValueError(f"Unknown node type {self.type}")

    def get_files(self, only_exists: bool = True, recursive=False) -> FSNode:
        """Get all files in the file structure as a `FSNode` objects under root `FSNode` object.

        Parameters
        ----------
        only_exists : bool, optional
            whether to include only existing files, that are marked as existing in the file structure after parsing, by default True
            WARNING: this method works only after parsing files with `parse_files` method

        Returns
        -------
        FSNode
            root `FSNode` object with all files in the flat file structure as children
        """
        files = {}
        self._get_files(
            files, only_exists, max_recursion_depth=(999 if recursive else 1)
        )
        root_node = FSNode(
            name="root_node",
            description=f"All files in file structure {'excluding' if only_exists else 'including'} non existing files",
            type="group",
            children=files,
        )
        # TODO[HIGH]: Add caching of processed files
        return root_node

    def __getitem__(self, key: str | None | Path) -> FSNode:
        """Get a child node by name hierarchically by using a list of keys

        Parameters
        ----------
        key : Iterifiable[str]
            complex key, e.g. ["group1", "file1"], may be a single key, e.g. "group1" or even `None`, which returns the root node

        Returns
        -------
        FSNode
            node with the given key

        Raises
        ------
        KeyError
            If the node with the given key does not exist.
        KeyError
            If the node has no children.
        """
        if key is None:
            return self

        path_key = Path(key)
        # absolute path
        absolute_key = (str(path_key.parent), path_key.stem)
        if absolute_key in self._children:
            return self._children[absolute_key]
        # relative path
        else:
            return self._children[(".", path_key.parts[0])][Path(*path_key.parts[1:])]

    def items(self):
        return self._children.items()

## Examples


Getting a list of paths from Google Cloud Storage


In [ ]:
browser = GSBrowser(project=project["project"], credentials=google_app_creds)
assert browser.is_present(RECSLAM_DATA_PATH), "Data is not present"

In [ ]:
remote_files = [Path(p.path).name for p in browser.list(RECSLAM_DATA_PATH)["files"]]

Initialize file structure from a json file.


In [ ]:
# | hide

RECSLAM_STRUCTURE

{'desc': 'Recslam file structure',
 'nodes': {'common': {'desc': 'All the information about device, not specific to certain camera',
   'nodes': {'detections': {'desc': 'Detections of different distresses and objects in the frame processed by client device',
     'path': 'detections.json'},
    'device': {'desc': 'Device information, camera params, etc.',
     'path': 'device.txt'},
    'gps': {'desc': 'GPS information from the device', 'path': 'gps.csv'},
    'heading': {'desc': 'Heading (direction) information from the device',
     'path': 'heading.csv'},
    'motion': {'desc': 'Motion information from the device, e.g. acceleration, rotation, etc.',
     'path': 'motion.csv'},
    'snapshots': {'desc': 'Snapshots of the video for composition',
     'path': 'snapshots.zip'}}},
  'camera_wide': {'desc': 'Files related to wide camera (_2 suffix) - main camera with higher quality and lower fps',
   'nodes': {'video': {'desc': 'Video captured by the wide camera (main camera with higher q

#### Examination of the file structure

Starting from the root of the fs.


In [ ]:
fs = FSNode.from_dict(RECSLAM_STRUCTURE)

Choose some group by accessing the group's name attribute


In [ ]:
fs["camera_wide"]

FSNode: [camera_wide]
{('.', 'video'): {'path': 'video_2', 'description': 'Video captured by the wide camera (main camera with higher quality and lower fps)', 'type': 'file', 'exists': None}, ('.', 'timestamps'): {'path': 'times_full_2.json', 'description': 'Timestamps of frames captured by the wide camera', 'type': 'file', 'exists': None}, ('.', 'timestamps_old'): {'path': 'times_2.txt', 'description': 'Timestamps of frames captured by the wide camera (old format)', 'type': 'file', 'exists': None}, ('.', 'raw_data'): {'path': '^video_data_\\d+\\.data_2$', 'description': 'Raw data captured by the wide camera', 'type': 'pattern', 'exists': None}}

We could use hierarchical keys to get values from a hierarchical file structure


In [ ]:
fs["camera_wide/video"]

,,path,description,type,exists
prefix,name,,,,
camera_wide,video,video_2,Video captured by the wide camera (main camera...,file,None


We see a pattern of the paths, it is not parsed yet. We could do it later.


#### Parsing list of real files


We could parse a real list of files, even virtual (non-existing), this will assign `exists` field to each file and parse `pattern` nodes.


In [ ]:
fs.parse_files(remote_files)

set()

In [ ]:
fs["camera_wide"]

FSNode: [camera_wide]
{('.', 'video'): {'path': 'video_2', 'description': 'Video captured by the wide camera (main camera with higher quality and lower fps)', 'type': 'file', 'exists': True}, ('.', 'timestamps'): {'path': 'times_full_2.json', 'description': 'Timestamps of frames captured by the wide camera', 'type': 'file', 'exists': True}, ('.', 'timestamps_old'): {'path': 'times_2.txt', 'description': 'Timestamps of frames captured by the wide camera (old format)', 'type': 'file', 'exists': True}, ('.', 'raw_data'): {'path': '^video_data_\\d+\\.data_2$', 'description': 'Raw data captured by the wide camera', 'type': 'pattern', 'exists': None}}

We could get a file itself. Look at the top of the representation, where FSNode is represented as `FSNode: [$type] $name: $description` and internal content


In [ ]:
fs["camera_wide/raw_data/video_data_23"]

,,path,description,type,exists
prefix,name,,,,
camera_wide/raw_data,video_data_23,video_data_23.data_2,Batch of raw video data,file,True


#### Get flat files representation

It may be used to parse only the files that are needed for the analysis. We could get a flat structure of all files (only files without other types of nodes) inside parsed file structure.


In [ ]:
files = fs.get_files(only_exists=True, recursive=True)
files

FSNode: [root_node]
{('common', 'device'): {'path': 'device.txt', 'description': 'Device information, camera params, etc.', 'type': 'file', 'exists': True}, ('common', 'gps'): {'path': 'gps.csv', 'description': 'GPS information from the device', 'type': 'file', 'exists': True}, ('common', 'heading'): {'path': 'heading.csv', 'description': 'Heading (direction) information from the device', 'type': 'file', 'exists': True}, ('common', 'motion'): {'path': 'motion.csv', 'description': 'Motion information from the device, e.g. acceleration, rotation, etc.', 'type': 'file', 'exists': True}, ('common', 'snapshots'): {'path': 'snapshots.zip', 'description': 'Snapshots of the video for composition', 'type': 'file', 'exists': True}, ('camera_wide', 'video'): {'path': 'video_2', 'description': 'Video captured by the wide camera (main camera with higher quality and lower fps)', 'type': 'file', 'exists': True}, ('camera_wide', 'timestamps'): {'path': 'times_full_2.json', 'description': 'Timestamps of frames captured by the wide camera', 'type': 'file', 'exists': True}, ('camera_wide', 'timestamps_old'): {'path': 'times_2.txt', 'description': 'Timestamps of frames captured by the wide camera (old format)', 'type': 'file', 'exists': True}, ('camera_wide/raw_data', 'video_data_13'): {'path': 'video_data_13.data_2', 'description': 'Batch of raw video data', 'type': 'file', 'exists': True}, ('camera_wide/raw_data', 'video_data_9'): {'path': 'video_data_9.data_2', 'description': 'Batch of raw video data', 'type': 'file', 'exists': True}, ('camera_wide/raw_data', 'video_data_4'): {'path': 'video_data_4.data_2', 'description': 'Batch of raw video data', 'type': 'file', 'exists': True}, ('camera_wide/raw_data', 'video_data_14'): {'path': 'video_data_14.data_2', 'description': 'Batch of raw video data', 'type': 'file', 'exists': True}, ('camera_wide/raw_data', 'video_data_10'): {'path': 'video_data_10.data_2', 'description': 'Batch of raw video data', 'type': 'file', 'exists': True}, ('camera_wide/raw_data', 'video_data_21'): {'path': 'video_data_21.data_2', 'description': 'Batch of raw video data', 'type': 'file', 'exists': True}, ('camera_wide/raw_data', 'video_data_23'): {'path': 'video_data_23.data_2', 'description': 'Batch of raw video data', 'type': 'file', 'exists': True}, ('camera_wide/raw_data', 'video_data_22'): {'path': 'video_data_22.data_2', 'description': 'Batch of raw video data', 'type': 'file', 'exists': True}, ('camera_wide/raw_data', 'video_data_3'): {'path': 'video_data_3.data_2', 'description': 'Batch of raw video data', 'type': 'file', 'exists': True}, ('camera_wide/raw_data', 'video_data_5'): {'path': 'video_data_5.data_2', 'description': 'Batch of raw video data', 'type': 'file', 'exists': True}, ('camera_wide/raw_data', 'video_data_25'): {'path': 'video_data_25.data_2', 'description': 'Batch of raw video data', 'type': 'file', 'exists': True}, ('camera_wide/raw_data', 'video_data_6'): {'path': 'video_data_6.data_2', 'description': 'Batch of raw video data', 'type': 'file', 'exists': True}, ('camera_wide/raw_data', 'video_data_11'): {'path': 'video_data_11.data_2', 'description': 'Batch of raw video data', 'type': 'file', 'exists': True}, ('camera_wide/raw_data', 'video_data_15'): {'path': 'video_data_15.data_2', 'description': 'Batch of raw video data', 'type': 'file', 'exists': True}, ('camera_wide/raw_data', 'video_data_2'): {'path': 'video_data_2.data_2', 'description': 'Batch of raw video data', 'type': 'file', 'exists': True}, ('camera_wide/raw_data', 'video_data_17'): {'path': 'video_data_17.data_2', 'description': 'Batch of raw video data', 'type': 'file', 'exists': True}, ('camera_wide/raw_data', 'video_data_24'): {'path': 'video_data_24.data_2', 'description': 'Batch of raw video data', 'type': 'file', 'exists': True}, ('camera_wide/raw_data', 'video_data_8'): {'path': 'video_data_8.data_2', 'description': 'Batch of raw video data', 'type': 'file', 'exists': True}, ('camera_wide/raw_data', 'video_data_12'): {'path': 

In [ ]:
files["common/device"]

,,path,description,type,exists
prefix,name,,,,
common,device,device.txt,"Device information, camera params, etc.",file,True


In [ ]:
fs["common"].get_files(only_exists=True, recursive=True)

FSNode: [root_node]
{('common', 'device'): {'path': 'device.txt', 'description': 'Device information, camera params, etc.', 'type': 'file', 'exists': True}, ('common', 'gps'): {'path': 'gps.csv', 'description': 'GPS information from the device', 'type': 'file', 'exists': True}, ('common', 'heading'): {'path': 'heading.csv', 'description': 'Heading (direction) information from the device', 'type': 'file', 'exists': True}, ('common', 'motion'): {'path': 'motion.csv', 'description': 'Motion information from the device, e.g. acceleration, rotation, etc.', 'type': 'file', 'exists': True}, ('common', 'snapshots'): {'path': 'snapshots.zip', 'description': 'Snapshots of the video for composition', 'type': 'file', 'exists': True}}

We could acces the properties of the FSNode directly


In [ ]:
files["common/device"].path, files["common/device"].description

('device.txt', 'Device information, camera params, etc.')

## GSBrowser File Structure


In [ ]:
downloads_dir = Directory("downloads", temporary=True)

In [ ]:
# | export


RemotePath = Path
LocalPath = Path


class GSBrowserFileStructure:
    def __init__(
        self,
        browser: GSBrowser,
        downloads_dir: Directory,
        file_structure_path: PathLike,
        fs_remote_root: GSBrowserContext,
    ) -> None:
        self._downloads_dir = downloads_dir
        self._file_structure: FSNode = self._init_file_structure(file_structure_path)
        self._flat_file_structure: FSNode | None = None
        self._browser = browser
        self._remote_root = fs_remote_root
        self._initialized = False
        self._initialize()

    def _initialize(self):
        logger.debug("Initializing GSBrowserFileStructure")
        self._init_list_file_structre()
        self._flat_file_structure: FSNode = self._file_structure.get_files(
            only_exists=False, recursive=True
        )
        self._init_attach_meta()
        logger.debug("Finished GSBrowserFileStructure")

    def _init_file_structure(self, file_structure_path):
        file_structure_path = pathify(file_structure_path)
        with open(file_structure_path, "r") as f:
            file_structure_dict = json.load(f)
        return FSNode.from_dict(file_structure_dict)

    def _init_list_file_structre(self):
        self._browser.cd(self._remote_root)
        logger.debug("Parsing remote files")
        remote_files = [
            Path(p.path).name for p in self._browser.list(self._remote_root)["files"]
        ]
        self._file_structure.parse_files(remote_files)

    def _init_attach_meta(self):
        for _, fsnode in self._flat_file_structure.items():
            if fsnode.exists:
                # attach local path
                fsnode.attach_meta(
                    "local_path",
                    Path(
                        *filter(
                            lambda x: x,
                            [
                                self._downloads_dir.path,
                                fsnode.path,
                            ],
                        )
                    ),
                )
                # attach cached
                fsnode.attach_meta("cached", fsnode.meta["local_path"].exists())

    def _repr_html_(self):
        return self._file_structure._repr_html_()

    def _get_file(
        self,
        fs_node: FSNode,
        download: bool = True,
        existing_handling_strategy: Literal["skip", "overwrite", "raise"] = "skip",
    ):
        if fs_node.meta.get("cached"):
            path = handle_existing_path(
                fs_node.meta["local_path"], existing_handling_strategy
            )
            return path
        else:
            local_path: Path | None = fs_node.meta.get("local_path")
            if local_path is None:
                raise ValueError(f"Local path is not defined for {fs_node}")
            else:
                if download:
                    self._browser.download_file(
                        fs_node.path,
                        local_path,
                        existing_handling_strategy,
                    )
                    fs_node.meta["cached"] = True
                    return local_path
                else:
                    raise KeyError(
                        f"File {fs_node} is not cached and download is set to False"
                    )

    def _get_group(self, fs_node: FSNode, download: bool, existing_handling, recursive):
        files = fs_node.get_files(only_exists=True, recursive=recursive)
        logger.info(
            f"Found {len(files.items())} files in {fs_node.path}, downloading..."
        )
        for _, file in files.items():
            yield self._get_file(file, download, existing_handling)

    def get(
        self,
        key: str | None,
        download=True,
        existing_handling: Literal["skip", "overwrite", "raise"] = "skip",
        recursive=False,
    ) -> list[Path | None]:
        """Gets files from the file structure, downloads them if necessary

        Parameters
        ----------
        key : str | None
            key of the file or group of files to get, by default None
        download : bool, optional
            whether to download the files if they are not cached, by default True
        existing_handling : Literal[&quot;skip&quot;, &quot;overwrite&quot;, &quot;raise&quot;], optional
            how to handle existing files, by default &quot;skip&quot;
        recursive : bool, optional
            whether to get files recursively, by default False

        Returns
        -------
        list[Path | None]
            list of paths to the files
        """
        fs_node = self._file_structure[key]
        if fs_node.type == "file":
            return [self._get_file(fs_node, download, existing_handling)]
        else:
            return [
                p
                for p in self._get_group(
                    fs_node, download, existing_handling, recursive
                )
            ]

    def __getitem__(self, key):
        return self._file_structure[key]

In [ ]:
dfs = GSBrowserFileStructure(
    browser,
    downloads_dir,
    CONFIGS_DIR / "storage/recslam/recslam_structure.json",
    RECSLAM_DATA_PATH,
)

In [ ]:
dfs["camera_wide/video"]

,,path,description,type,exists,meta[local_path],meta[cached]
prefix,name,,,,,,
camera_wide,video,video_2,Video captured by the wide camera (main camera...,file,True,/Users/arseniy/Projects/dev/ds_contrib/nbs/cor...,False


We can get a single file


In [ ]:
# |eval: false
dfs.get("camera_wide/video")

[Path('/Users/arseniy/Projects/dev/ds_contrib/nbs/core/downloads/tmp3n32u3sa/video_2')]

Or the whole logical group of files


In [ ]:
# |eval: false
dfs.get("common")

[Path('/Users/arseniy/Projects/dev/ds_contrib/nbs/core/downloads/tmp3n32u3sa/device.txt'),
 Path('/Users/arseniy/Projects/dev/ds_contrib/nbs/core/downloads/tmp3n32u3sa/gps.csv'),
 Path('/Users/arseniy/Projects/dev/ds_contrib/nbs/core/downloads/tmp3n32u3sa/heading.csv'),
 Path('/Users/arseniy/Projects/dev/ds_contrib/nbs/core/downloads/tmp3n32u3sa/motion.csv'),
 Path('/Users/arseniy/Projects/dev/ds_contrib/nbs/core/downloads/tmp3n32u3sa/snapshots.zip')]

In [ ]:
downloads_dir.cleanup()

---


In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()